# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key

In [16]:
!jupyter nbextension enable --py gmaps
!jupyter nbextension enable --py widgetsnbextension
%matplotlib 

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


Using matplotlib backend: Qt5Agg


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
file = "../output_data/cities_weather.csv"

df = pd.read_csv(file)

df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Denpasar,75,ID,1611523464,94,-8.6500,115.2167,78.80,1.14
1,Kapaa,1,US,1611523515,73,22.0752,-159.3190,78.80,13.80
2,Nabire,100,ID,1611523435,90,-3.3667,135.4833,75.29,5.57
3,Mampong,0,GH,1611523739,26,7.0627,-1.4001,77.32,4.05
4,Novopokrovka,0,UA,1611523739,75,49.8425,36.5477,35.60,11.18
...,...,...,...,...,...,...,...,...,...
565,Jishou,100,CN,1611523824,90,28.3167,109.7167,46.31,1.52
566,Qaqortoq,0,GL,1611523824,31,60.7167,-46.0333,26.60,36.82
567,Alexandria,40,EG,1611523824,67,31.2156,29.9553,59.00,4.61
568,Alibag,1,IN,1611523824,56,18.6411,72.8792,73.40,4.61


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [45]:
#Configure gmaps
gmaps.configure(api_key = g_key)

locations = df[["Lat", "Lng"]]

weight = df['Humidity']

weight


0      94
1      73
2      90
3      26
4      75
       ..
565    90
566    31
567    67
568    56
569    93
Name: Humidity, Length: 570, dtype: int64

In [47]:
fig = gmaps.figure(center = (10, 0), zoom_level = 1.8)
fig.add_layer(gmaps.heatmap_layer(locations, weights = weight, dissipating = False))

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [50]:
narrow_df = df.loc[(df["Wind Speed"] <= 12) & (df["Cloudiness"] <= 10) & 
                   (df["Max Temp"] >= 68) & (df["Max Temp"] <= 78) & (df["Humidity"] <= 51)].dropna()

narrow_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
3,Mampong,0,GH,1611523739,26,7.0627,-1.4001,77.32,4.05
8,Hobart,0,AU,1611523442,40,-42.8794,147.3294,70.00,8.05
67,New Norfolk,5,AU,1611523506,37,-42.7826,147.0587,73.00,1.01
309,Abéché,9,TD,1611523781,18,13.8292,20.8324,75.20,11.32
464,Salalah,0,OM,1611523548,37,17.0151,54.0924,71.60,6.91
475,Ouallam,0,NE,1611523558,17,14.3188,2.0866,72.70,8.79
550,Ségou,0,ML,1611523822,18,13.4317,-6.2157,71.91,9.78


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [54]:
hotel_df = narrow_df

#Adding Hotel Name
hotel_df["Hotel Name"] = ""

hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
3,Mampong,0,GH,1611523739,26,7.0627,-1.4001,77.32,4.05,
8,Hobart,0,AU,1611523442,40,-42.8794,147.3294,70.00,8.05,
67,New Norfolk,5,AU,1611523506,37,-42.7826,147.0587,73.00,1.01,
309,Abéché,9,TD,1611523781,18,13.8292,20.8324,75.20,11.32,
464,Salalah,0,OM,1611523548,37,17.0151,54.0924,71.60,6.91,
475,Ouallam,0,NE,1611523558,17,14.3188,2.0866,72.70,8.79,
550,Ségou,0,ML,1611523822,18,13.4317,-6.2157,71.91,9.78,


In [65]:
#Creating base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Creating parameters dictionary for quickness ease
params = {"type" : "hotel",
         "keyword" : "hotel",
         "radius" : 5000,
         "key" : g_key}

#Have base info for making API call, now need to build loop
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    country_output = row["Country"]
    
    #Adding locations to params dictionary
    params["location"] = f"{lat},{lng}"
    
    #Building query URL
    query_url = requests.get(base_url, params=params)
    query_url = query_url.json()
    
    #Output storage
    output = query_url["results"]
    
    try:
        #Creating print out for sanity check
        print(f"Closest hotel in {city_name}, {country_output} is: {output[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = output[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------------------------")
    
print("-----Query completed successfully.-----")

Closest hotel in Mampong, GH is: DAAHO ROYAL HOTEL.
------------------------------
Closest hotel in Hobart, AU is: Hotel Grand Chancellor Hobart.
------------------------------
Missing field/result... skipping.
------------------------------
Closest hotel in Abéché, TD is: Hôtel kinpeski.
------------------------------
Closest hotel in Salalah, OM is: HYATT SALALAH HOTEL.
------------------------------
Missing field/result... skipping.
------------------------------
Closest hotel in Ségou, ML is: Hotel Independance.
------------------------------
-----Query completed successfully.-----


In [62]:
#Viewing hotel dataframe
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
3,Mampong,0,GH,1611523739,26,7.0627,-1.4001,77.32,4.05,DAAHO ROYAL HOTEL
8,Hobart,0,AU,1611523442,40,-42.8794,147.3294,70.00,8.05,Vibe Hotel Hobart
67,New Norfolk,5,AU,1611523506,37,-42.7826,147.0587,73.00,1.01,
309,Abéché,9,TD,1611523781,18,13.8292,20.8324,75.20,11.32,Hôtel kinpeski
464,Salalah,0,OM,1611523548,37,17.0151,54.0924,71.60,6.91,HYATT SALALAH HOTEL
475,Ouallam,0,NE,1611523558,17,14.3188,2.0866,72.70,8.79,
550,Ségou,0,ML,1611523822,18,13.4317,-6.2157,71.91,9.78,Hotel Independance


In [63]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [64]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))